# Esperanza's Development (2nd) Notebook
This notebook is from before the re-cleaning: prompted some of the re-cleaning work available in that notebook.

## Imports

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

import prophet as Prophet

In [ ]:
# Prophet import gives: "Importing plotly failed. Interactive plots will not work."
# Was not able to solve this

## Grabbing Data from Data/Out

In [ ]:
#col_list = ['time', 'Common Setpoint', 'Actual Sup Flow SP', 'Zone Temperature', 'Actual Supply Flow', 'Outside Air Temp', 'Humidity', 'energy']

floor_2 = pd.read_csv('../data/raw/merged_all_2F.csv', index_col = 'Unnamed: 0')
floor_3 = pd.read_csv('../data/raw/merged_all_3F.csv', index_col = 'Unnamed: 0')
floor_4 = pd.read_csv('../data/raw/merged_all_4F.csv', index_col = 'Unnamed: 0')

In [ ]:
# one version of this is Unnamed: 0.2 but I'm not sure why this one says 

floor_2 = floor_2.drop(['Unnamed: 0.1.1'], axis = 1)

In [ ]:
floor_2.head(5)

In [ ]:
floor_2['time_transformed'] = pd.to_datetime(floor_2['time'])
floor_2['time_floored'] = floor_2['time_transformed'].transform(lambda x: x.floor(freq='5T'))
floor_2['unix_floored'] = floor_2['time_floored'].view('int64')

### Trying to Figure Out Room Counts based on Times

In [ ]:
floor_2['time_floored'].value_counts().head(5)

In [ ]:
floor_2['time_floored'].value_counts().tail(5)

In [ ]:
floor_2.groupby(['time_floored', 'Unnamed: 0.1']).count()['time'].head(5)

In [ ]:
floor_2.groupby(['time_floored', 'Unnamed: 0.1']).count()['time'].tail(5)

In [ ]:
floor_2.groupby(['time_floored', 'Unnamed: 0.1']).count()['time'].max()

In [ ]:
floor_2.groupby(['time_floored', 'Unnamed: 0.1']).count()['time'].min()

In [ ]:
# round timestamps: either floor or higher consistently for timestamps --> explain assumption

# transform to unix timestamp*** check with prophet compatibility

In [ ]:
#floor_3 = floor_3.drop(['Unnamed: 0.1'], axis = 1)

In [ ]:
floor_3.head(5)

In [ ]:
floor_3['time_transformed'] = pd.to_datetime(floor_3['time'])
floor_3['time_floored'] = floor_3['time_transformed'].transform(lambda x: x.floor(freq='5T'))
floor_3['unix_floored'] = floor_3['time_floored'].view('int64')

In [ ]:
floor_3['time_floored'].value_counts().head(5)

In [ ]:
floor_3['time_floored'].value_counts().tail(5)

In [ ]:
floor_3.groupby(['time_floored', 'Unnamed: 0.1']).count()['time'].head(5)

In [ ]:
floor_3.groupby(['time_floored', 'Unnamed: 0.1']).count()['time'].tail(5)

In [ ]:
floor_3.groupby(['time_floored', 'Unnamed: 0.1']).count()['time'].max()

In [ ]:
floor_3.groupby(['time_floored', 'Unnamed: 0.1']).count()['time'].min()

Attempting to build 3 different models for each floor instead of trying a combination of all the data.

In [ ]:
#floor_4 = floor_4.drop(['Unnamed: 0.1'], axis = 1)

In [ ]:
floor_4.head(5)

In [ ]:
floor_4['time_transformed'] = pd.to_datetime(floor_4['time'])
floor_4['time_floored'] = floor_4['time_transformed'].transform(lambda x: x.floor(freq='5T'))
floor_4['unix_floored'] = floor_4['time_floored'].view('int64')

In [ ]:
floor_4['time_floored'].value_counts().head(5)

In [ ]:
floor_4['time_floored'].value_counts().tail(5)

In [ ]:
floor_4.groupby(['time_floored', 'Unnamed: 0.1']).count()['time'].head(5)

In [ ]:
floor_4.groupby(['time_floored', 'Unnamed: 0.1']).count()['time'].tail(5)

In [ ]:
# floor 4 is more regular but it's hard to tell what's going on here

In [ ]:
floor_4.groupby(['time_floored', 'Unnamed: 0.1']).count()['time'].max()

In [ ]:
floor_4.groupby(['time_floored', 'Unnamed: 0.1']).count()['time'].min()

In [ ]:
# 5 rooms on floor 4, 4 on floor 3, 6 on floor 2
# how is it worth doing averages here?
# do we try and impute with the mean value for the floor and the timestamp?
# doesn't fix the issue that the main time data is different

### Combining Data

In [ ]:
floor_2['floor'] = 2
floor_3['floor'] = 3
floor_4['floor'] = 4

In [ ]:
combined_floors = pd.concat([floor_2, floor_3, floor_4]).reset_index(drop = True)

In [ ]:
combined_floors.head(5)

In [ ]:
mean_energy = combined_floors.groupby(['time_floored', 'Unnamed: 0.1']).mean()['energy']

In [ ]:
counts = combined_floors.groupby(['time_floored', 'Unnamed: 0.1']).count()['energy'].rename('counts')

In [ ]:
counts.to_frame().head(5)

In [ ]:
counts.to_frame().tail(5)

In [ ]:
need_more_vals = 6 - counts[6 - counts > 0].to_frame()

In [ ]:
need_more_vals.shape

In [ ]:
merged_duplicates = need_more_vals.merge(mean_energy.to_frame(), left_index = True, right_index = True)

In [ ]:
group_1 = merged_duplicates.reset_index().drop(['counts'], axis = 1)

In [ ]:
group_2 = merged_duplicates.loc[merged_duplicates['counts'] >= 2].reset_index().drop(['counts'], axis = 1)
group_3 = merged_duplicates.loc[merged_duplicates['counts'] >= 3].reset_index().drop(['counts'], axis = 1)
group_4 = merged_duplicates.loc[merged_duplicates['counts'] >= 4].reset_index().drop(['counts'], axis = 1)
group_5 = merged_duplicates.loc[merged_duplicates['counts'] == 5].reset_index().drop(['counts'], axis = 1)

In [ ]:
relevant_vals = combined_floors.loc[:, ['time_floored', 'energy', 'Unnamed: 0.1']]

In [ ]:
total_group = pd.concat([relevant_vals, group_1, group_2, group_3, group_4, group_5])

In [ ]:
counts.to_frame().value_counts()

In [ ]:
relevant_vals.shape

In [ ]:
group_1.shape

In [ ]:
total_group.shape

In [ ]:
total_group.groupby(['time_floored', 'Unnamed: 0.1']).count()['energy'].value_counts()

In [ ]:
combined_floors.groupby(['time_floored', 'Unnamed: 0.1']).count()['energy'].min()

In [ ]:
# Do we fill this up to 6 with the mean value and see what that does?

In [ ]:
combined_merge = combined_floors.merge(counts.to_frame(), left_on = ['time_floored', 'Unnamed: 0.1'], right_index = True)

In [ ]:
combined_merge.head()

In [ ]:
timestamp_df = counts.to_frame()

In [ ]:
df_ver6 = pd.concat([timestamp_df] * 6).reset_index()

In [ ]:
df_ver6.head()

In [ ]:
df_ver6.shape

In [ ]:
combined_merge_2 = combined_floors.merge(df_ver6, how = 'outer', left_on = ['time_floored', 'Unnamed: 0.1'], right_on = ['time_floored', 'Unnamed: 0.1'])

In [ ]:
combined_merge_2.head(5)

In [ ]:
combined_merge_2['energy'].isna().sum()

In [ ]:
combined_merge_2.drop_duplicates().shape

In [ ]:
combined_merge.shape

In [ ]:
combined_merge[combined_merge['counts'] < 6].shape

## Getting Relevant Dates for Train / Test Data

In [ ]:
dates = combined_floors['time_transformed'].transform(lambda x: x.date)

In [ ]:
dates_test = (dates >= pd.Timestamp('2018-08-01'))

In [ ]:
onwards_test_date = combined_floors.loc[dates_test, :]
pre_test_date = combined_floors.loc[~dates_test, :]

In [ ]:
pre_test_date.shape

In [ ]:
onwards_test_date.shape

## Work with Prophet

#### For missing analysis work, please look at old version of this notebook.

In [ ]:
# choosing the relevant info
pre_test_date_reduced = pre_test_date.loc[:, ['time', 'energy']].reset_index(drop = True)
onwards_test_date_reduced = onwards_test_date.loc[:, ['time', 'energy']].reset_index(drop = True)

In [ ]:
# stripping time zones since they're incompatible with Prophet
pre_test_date_reduced['time'] = pre_test_date_reduced['time'].transform(lambda x: x[0:-6])
onwards_test_date_reduced['time'] = onwards_test_date_reduced['time'].transform(lambda x: x[0:-6])

In [ ]:
pre_test_date_reduced = pre_test_date_reduced.rename({'time': 'ds', 'energy': 'y'}, axis = 1)
onwards_test_date_reduced = onwards_test_date_reduced.rename({'time': 'ds', 'energy': 'y'}, axis = 1)


### Determining cap for Prophet model

In [ ]:
pre_test_date_reduced['y'].sort_values(ascending = False).head(5)

In [ ]:
pre_test_date_reduced['y'].mean()

In [ ]:
pre_test_date_reduced['y'].median()

In [ ]:
pre_test_date_reduced['cap'] = 50
# have not set a changing cap with HVAC because I'm not sure it did change over this time frame

### Training Model

In [ ]:
prophet_model = Prophet.Prophet()

In [ ]:
prophet_model.fit(pre_test_date_reduced)
# got disabling yearly seasonality text - I think this makes sense to disable because our data < year

### Making Model Forecast

#### We want to predict every 5 minutes from August 2, 2018 through the end of the data on January 7?, 2019
This involves 5 minutes ... still figuring this out

In [ ]:
prophet_forecast = prophet_model.make_future_dataframe(periods = 51000, freq = '5min')
prophet_forecast = prophet_model.predict(prophet_forecast)


In [ ]:
prophet_forecast.tail(5)

In [ ]:
# This got us a little past the time I wanted to be at but it works

## Plot Analysis

#### Initial result shows that y hat values grow too quickly

In [ ]:
fig2 = prophet_model.plot_components(prophet_forecast)

In [ ]:
#Regular for days of the week and maybe for hours, but not for overall months


In [ ]:
prophet_forecast_reduced = prophet_forecast.loc[:, ['ds', 'yhat']]

In [ ]:
prophet_forecast_reduced['timestamp_changed'] = prophet_forecast_reduced['ds'].transform(lambda x: pd.Timestamp(x))

In [ ]:
#onwards_compare = onwards_test_date.loc[:, ['time', 'energy']]
#onwards_compare['time'] = onwards_compare['time'].transform(lambda x: x[0:-6])
#onwards_compare = onwards_compare.rename({'time': 'ds', 'energy': 'yhat'}, axis = 1)
onwards_test_date_reduced['timestamp_changed'] = onwards_test_date_reduced['ds'].transform(lambda x: pd.Timestamp(x))

In [ ]:
onwards_test_date_reduced.head(5)

In [ ]:
onwards_test_date_reduced.shape

In [ ]:
pre_test_date_reduced.shape

In [ ]:
merge_test = prophet_forecast_reduced.merge(onwards_test_date_reduced, left_on = 'timestamp_changed', right_on = 'timestamp_changed')

In [ ]:
merge_test.head(5)

In [ ]:
merge_test['yhat'].nunique() # there's only 77 unique values without using the floored values

In [ ]:
no_duplicates = merge_test.drop_duplicates(subset=['ds_x', 'yhat'], keep = 'first')

In [ ]:
no_duplicates.head(5)

In [ ]:
no_duplicates.shape

In [ ]:
onwards_test_date_reduced.loc[onwards_test_date_reduced['timestamp_changed'] == pd.Timestamp('2018-08-19T23:47:40')]

In [ ]:
# this dataset introduces duplicate measurements from each of the floors but it also has multiple duplicates

# per floor -- should we train on each or how should we do this?

In [ ]:
# I'm very confused why there's duplicates here??? Need to clear these up and see which y values are causing this

### Attempt #2 - Working with floor 2 alone

In [ ]:
floor_2_reduced = floor_2.loc[:, ['time', 'energy']].rename({'time': 'ds', 'energy': 'y'}, axis = 1)

In [ ]:
floor_2_reduced['ds'] = floor_2_reduced['ds'].str[0:-6]

In [ ]:
floor_2_reduced['time_transformed'] = floor_2_reduced['ds'].transform(pd.Timestamp)

In [ ]:
shape_val = floor_2_reduced['time_transformed'].shape[0]

In [ ]:
np.round(((0.70) * shape_val))

In [ ]:
seventy_front = floor_2_reduced.sort_values(by='time_transformed')[0:int(np.round(((0.70) * shape_val)))]
thirty_back = floor_2_reduced.sort_values(by='time_transformed')[int(np.round(((0.70) * shape_val))):]

In [ ]:
seventy_front_red = seventy_front.drop(['time_transformed'], axis = 1)
thirty_back_red = thirty_back.drop(['time_transformed'], axis = 1)

In [ ]:
thirty_back.head()

In [ ]:
prophet_model = Prophet.Prophet()

In [ ]:
prophet_model.fit(seventy_front_red)


In [ ]:
prophet_forecast = prophet_model.make_future_dataframe(periods = 51000, freq = '5min')
prophet_forecast = prophet_model.predict(prophet_forecast)

In [ ]:
prophet_forecast.tail(5)

In [ ]:
fig2 = prophet_model.plot_components(prophet_forecast)

In [ ]:
# so this actually seems pretty similar to the work before even though numbers are different
# let's see if we can do evalution

In [ ]:
prophet_forecast_times = prophet_forecast['ds'].transform(pd.Timestamp).rename('time_transformed')


In [ ]:
prophet_forecast_times.head(5)

In [ ]:
prophet_forecast_total = prophet_forecast.merge(prophet_forecast_times.to_frame(), left_index = True, right_index = True)

In [ ]:
prophet_forecast_total.head()

In [ ]:
compare = prophet_forecast_total.merge(thirty_back, how = 'outer', left_on = 'time_transformed', right_on = 'time_transformed')

In [ ]:
compare.head()

In [ ]:
compare.shape

In [ ]:
compare['ds_y'].dropna().head(5)

In [ ]:
compare_future = compare[33306:]

In [ ]:
compare_future.head()

In [ ]:
compare_future.shape

In [ ]:
compare_future.loc[:, ['yhat', 'y']].tail(5)

In [ ]:
compare_future_means = compare_future

In [ ]:
compare_future_means['yhat'] = compare_future_means['yhat'].fillna(compare_future_means['yhat'].mean())
compare_future_means['y'] = compare_future_means['y'].fillna(compare_future_means['y'].mean())

In [ ]:
mse_floor_2 = ((compare_future_means['yhat'] - compare_future_means['y']) ** 2).sum() / compare_future_means.shape[0]

In [ ]:
mse_floor_2

In [ ]:
# comparing just the unique values for comparison, MSE of 50 isn't actually awful --> let me try with other floors

### Attempt # 3 / Floor 3

In [ ]:
floor_3_reduced = floor_3.loc[:, ['time', 'energy']].rename({'time': 'ds', 'energy': 'y'}, axis = 1)

In [ ]:
floor_3_reduced['ds'] = floor_3_reduced['ds'].str[0:-6]

In [ ]:
floor_3_reduced['time_transformed'] = floor_3_reduced['ds'].transform(pd.Timestamp)

In [ ]:
shape_val = floor_3_reduced['time_transformed'].shape[0]

In [ ]:
np.round(((0.70) * shape_val))

In [ ]:
seventy_front = floor_3_reduced.sort_values(by='time_transformed')[0:int(np.round(((0.70) * shape_val)))]
thirty_back = floor_3_reduced.sort_values(by='time_transformed')[int(np.round(((0.70) * shape_val))):]

In [ ]:
seventy_front_red = seventy_front.drop(['time_transformed'], axis = 1)
thirty_back_red = thirty_back.drop(['time_transformed'], axis = 1)

In [ ]:
seventy_front.tail()

In [ ]:
prophet_model = Prophet.Prophet()

In [ ]:
prophet_model.fit(seventy_front_red)


In [ ]:
prophet_forecast = prophet_model.make_future_dataframe(periods = 51000, freq = '5min')
prophet_forecast = prophet_model.predict(prophet_forecast)

In [ ]:
prophet_forecast.tail(5)

In [ ]:
fig3 = prophet_model.plot_components(prophet_forecast)

In [ ]:
# so this actually seems pretty similar to the work before even though numbers are different
# let's see if we can do evaluation

In [ ]:
prophet_forecast_times = prophet_forecast['ds'].transform(pd.Timestamp).rename('time_transformed')


In [ ]:
prophet_forecast_times.head(5)

In [ ]:
prophet_forecast_total = prophet_forecast.merge(prophet_forecast_times.to_frame(), left_index = True, right_index = True)

In [ ]:
prophet_forecast_total.head()

In [ ]:
compare = prophet_forecast_total.merge(thirty_back, how = 'outer', left_on = 'time_transformed', right_on = 'time_transformed')

In [ ]:
compare.head()

In [ ]:
compare.shape

In [ ]:
compare['ds_y'].dropna()

In [ ]:
compare_future = compare[34534:]

In [ ]:
compare_future.head()

In [ ]:
compare_future.shape

In [ ]:
compare_future.loc[:, ['yhat', 'y']]

In [ ]:
compare_future_means = compare_future

In [ ]:
compare_future_means['yhat'] = compare_future_means['yhat'].fillna(compare_future_means['yhat'].mean())
compare_future_means['y'] = compare_future_means['y'].fillna(compare_future_means['y'].mean())

In [ ]:
mse_floor_3 = ((compare_future_means['yhat'] - compare_future_means['y']) ** 2).sum() / compare_future_means.shape[0]

In [ ]:
mse_floor_3

# Attempt # 4 / Floor 4

In [ ]:
floor_4_reduced = floor_4.loc[:, ['time', 'energy']].rename({'time': 'ds', 'energy': 'y'}, axis = 1)

In [ ]:
floor_4_reduced['ds'] = floor_4_reduced['ds'].str[0:-6]

In [ ]:
floor_4_reduced['time_transformed'] = floor_4_reduced['ds'].transform(pd.Timestamp)

In [ ]:
shape_val = floor_4_reduced['time_transformed'].shape[0]

In [ ]:
np.round(((0.70) * shape_val))

In [ ]:
seventy_front = floor_4_reduced.sort_values(by='time_transformed')[0:int(np.round(((0.70) * shape_val)))]
thirty_back = floor_4_reduced.sort_values(by='time_transformed')[int(np.round(((0.70) * shape_val))):]

In [ ]:
seventy_front_red = seventy_front.drop(['time_transformed'], axis = 1)
thirty_back_red = thirty_back.drop(['time_transformed'], axis = 1)


In [ ]:
seventy_front.tail()

In [ ]:
prophet_model = Prophet.Prophet()

In [ ]:
prophet_model.fit(seventy_front_red)


In [ ]:
prophet_forecast = prophet_model.make_future_dataframe(periods = 51000, freq = '5min')
prophet_forecast = prophet_model.predict(prophet_forecast)

In [ ]:
prophet_forecast.tail(5)

In [ ]:
fig4 = prophet_model.plot_components(prophet_forecast)

In [ ]:
# The values for dates change wildly here, comapared to the other 2 floors

In [ ]:
prophet_forecast_times = prophet_forecast['ds'].transform(pd.Timestamp).rename('time_transformed')


In [ ]:
prophet_forecast_times.head(5)

In [ ]:
prophet_forecast_total = prophet_forecast.merge(prophet_forecast_times.to_frame(), left_index = True, right_index = True)

In [ ]:
prophet_forecast_total.head()

In [ ]:
compare = prophet_forecast_total.merge(thirty_back, how = 'outer', left_on = 'time_transformed', right_on = 'time_transformed')

In [ ]:
compare.head()

In [ ]:
compare.shape

In [ ]:
compare['ds_y'].dropna().head(20)

In [ ]:
compare_future = compare[25368:]

In [ ]:
compare_future.head()

In [ ]:
compare_future.shape

In [ ]:
compare_future.loc[:, ['yhat', 'y']]

In [ ]:
compare_future_means = compare_future

In [ ]:
compare_future_means['yhat'] = compare_future_means['yhat'].fillna(compare_future_means['yhat'].mean())
compare_future_means['y'] = compare_future_means['y'].fillna(compare_future_means['y'].mean())

In [ ]:
mse_floor_4 = ((compare_future_means['yhat'] - compare_future_means['y']) ** 2).sum() / compare_future_means.shape[0]

In [ ]:
mse_floor_4

In [ ]:
# ok so this is interesting --> the main contributor to issues is the floor 4 data -- we can see this with the trends as well


In [ ]:
# Day of the week and time of day trends are pretty similar across 
# But for some reason prophet's struggling with the week 4 data

In [ ]:
#Is there a couple of values that are just really high or that spike after September 2018 to cause this?